In [1]:
from ift6758.data import load_train_test_dataframes

import pandas as pd

In [3]:
train_data, test_data = load_train_test_dataframes(True)

Found 413339 events
Found 434744 events
Found 433203 events
Found 367886 events
Found 287926 events


In [14]:
print("Features in the training data:")
train_data.columns

Features in the training data:


Index(['game_id', 'season', 'game_type', 'game_date', 'venue',
       'venue_location', 'away_team_id', 'away_team_abbrev', 'away_team_name',
       'home_team_id', 'home_team_abbrev', 'home_team_name', 'event_id',
       'event_idx', 'sort_order', 'period_number', 'period_type',
       'max_regulation_periods', 'time_in_period', 'time_remaining',
       'situation_code', 'is_empty_net', 'is_goal', 'type_code',
       'type_desc_key', 'away_score', 'home_score', 'away_sog', 'home_sog',
       'x_coord', 'y_coord', 'zone_code', 'shot_type', 'description',
       'event_owner_team_id', 'scoring_player_total', 'assist1_player_total',
       'assist2_player_total', 'goalie_in_net_id', 'goalie_in_net_name',
       'goalie_in_net_team_id', 'goalie_in_net_position_code',
       'shooting_player_id', 'shooting_player_name', 'shooting_player_team_id',
       'shooting_player_position_code', 'goal_distance', 'goal_angle',
       'goal_side', 'goal_x_coord', 'scoring_player_id', 'scoring_player_n

In [24]:
# i want to print a full row of the training data, no truncation
pd.set_option('display.max_columns', None)
print("First row of the training data:")
display(train_data.head(1))


First row of the training data:


,game_id,season,game_type,game_date,venue,venue_location,away_team_id,away_team_abbrev,away_team_name,home_team_id,home_team_abbrev,home_team_name,event_id,event_idx,sort_order,period_number,period_type,max_regulation_periods,time_in_period,time_remaining,situation_code,is_empty_net,is_goal,type_code,type_desc_key,away_score,home_score,away_sog,home_sog,x_coord,y_coord,zone_code,shot_type,description,event_owner_team_id,scoring_player_total,assist1_player_total,assist2_player_total,goalie_in_net_id,goalie_in_net_name,goalie_in_net_team_id,goalie_in_net_position_code,shooting_player_id,shooting_player_name,shooting_player_team_id,shooting_player_position_code,goal_distance,goal_angle,goal_side,goal_x_coord,scoring_player_id,scoring_player_name,scoring_player_team_id,scoring_player_position_code,assist1_player_id,assist1_player_name,assist1_player_team_id,assist1_player_position_code,assist2_player_id,assist2_player_name,assist2_player_team_id,assist2_player_position_code
0,2016020001,20162017,2,2016-10-12,Canadian Tire Centre,Ottawa,10,TOR,Maple Leafs,9,OTT,Senators,8,7,16,1,REG,3,01:11,18:49,1551,False,False,506,shot-on-goal,NaN,NaN,1.0,0.0,-77.0,5.0,O,wrist,Craig Anderson stops a shot from Mitch Marner,10,NaN,NaN,NaN,8467950.0,Craig Anderson,9.0,G,8478483.0,Mitch Marner,10.0,C,13.0,-22.619865,left,-89,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
print("Shape of the training data:", train_data.shape)
print("Shape of the testing data:", test_data.shape)

print("Columns that are in the training data but not in the testing data (target):")
print(set(train_data.columns) - set(test_data.columns))

Shape of the training data: (327342, 62)
Shape of the testing data: (57734, 61)
Columns that are in the training data but not in the testing data (Target):
{'is_goal'}


### Utils

In [25]:
# Convert time_in_period from MM:SS to seconds
def convert_to_seconds(time_str):
    minutes, seconds = map(int, time_str.split(':'))
    return minutes * 60 + seconds

### Preprocessing function (Advanced)

In [26]:
def preprocess_advanced(data):
    data['time_in_period_seconds'] = data['time_in_period'].apply(convert_to_seconds)
    
    # Compute game_seconds
    data['game_seconds'] = (data['period_number'] - 1) * 1200 + data['time_in_period_seconds']
    
    return data


In [27]:
preprocess_advanced(train_data)

train_data.head()

,game_id,season,game_type,game_date,venue,venue_location,away_team_id,away_team_abbrev,away_team_name,home_team_id,home_team_abbrev,home_team_name,event_id,event_idx,sort_order,period_number,period_type,max_regulation_periods,time_in_period,time_remaining,situation_code,is_empty_net,is_goal,type_code,type_desc_key,away_score,home_score,away_sog,home_sog,x_coord,y_coord,zone_code,shot_type,description,event_owner_team_id,scoring_player_total,assist1_player_total,assist2_player_total,goalie_in_net_id,goalie_in_net_name,goalie_in_net_team_id,goalie_in_net_position_code,shooting_player_id,shooting_player_name,shooting_player_team_id,shooting_player_position_code,goal_distance,goal_angle,goal_side,goal_x_coord,scoring_player_id,scoring_player_name,scoring_player_team_id,scoring_player_position_code,assist1_player_id,assist1_player_name,assist1_player_team_id,assist1_player_position_code,assist2_player_id,assist2_player_name,assist2_player_team_id,assist2_player_position_code,time_in_period_seconds,game_seconds
0,2016020001,20162017,2,2016-10-12,Canadian Tire Centre,Ottawa,10,TOR,Maple Leafs,9,OTT,Senators,8,7,16,1,REG,3,01:11,18:49,1551,False,False,506,shot-on-goal,NaN,NaN,1.0,0.0,-77.0,5.0,O,wrist,Craig Anderson stops a shot from Mitch Marner,10,NaN,NaN,NaN,8467950.0,Craig Anderson,9.0,G,8478483.0,Mitch Marner,10.0,C,13.000000,-22.619865,left,-89,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,71,71
1,2016020001,20162017,2,2016-10-12,Canadian Tire Centre,Ottawa,10,TOR,Maple Leafs,9,OTT,Senators,11,14,38,1,REG,3,02:53,17:07,1551,False,False,506,shot-on-goal,NaN,NaN,1.0,1.0,86.0,13.0,O,wrist,Frederik Andersen stops a shot from Chris Kelly,9,NaN,NaN,NaN,8475883.0,Frederik Andersen,10.0,G,8467967.0,Chris Kelly,9.0,C,13.341664,77.005383,right,89,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,173,173
2,2016020001,20162017,2,2016-10-12,Canadian Tire Centre,Ottawa,10,TOR,Maple Leafs,9,OTT,Senators,15,21,53,1,REG,3,04:01,15:59,1551,False,False,506,shot-on-goal,NaN,NaN,1.0,2.0,23.0,-38.0,N,wrist,Frederik Andersen stops a shot from Cody Ceci,9,NaN,NaN,NaN,8475883.0,Frederik Andersen,10.0,G,8476879.0,Cody Ceci,9.0,D,76.157731,-29.931512,left,89,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,241,241
3,2016020001,20162017,2,2016-10-12,Canadian Tire Centre,Ottawa,10,TOR,Maple Leafs,9,OTT,Senators,16,23,60,1,REG,3,04:46,15:14,1551,False,False,506,shot-on-goal,NaN,NaN,1.0,3.0,33.0,-15.0,O,slap,Frederik Andersen stops a shot from Erik Karlsson,9,NaN,NaN,NaN,8475883.0,Frederik Andersen,10.0,G,8474578.0,Erik Karlsson,9.0,D,57.974132,-14.995079,left,89,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,286,286
4,2016020001,20162017,2,2016-10-12,Canadian Tire Centre,Ottawa,10,TOR,Maple Leafs,9,OTT,Senators,24,36,86,1,REG,3,06:46,13:14,1551,False,False,506,shot-on-goal,NaN,NaN,2.0,3.0,-34.0,28.0,O,wrist,Craig Anderson stops a shot from Martin Marincin,10,NaN,NaN,NaN,8467950.0,Craig Anderson,9.0,G,8475716.0,Martin Marincin,10.0,D,61.717096,-26.980231,left,-89,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,406,406
